In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
from torchvision import datasets, transforms, models

In [2]:
train_transform = transforms.Compose([transforms.Resize((224, 224)),
                                      transforms.GaussianBlur(kernel_size=(7, 13), sigma=(0.5, 0.5)),
                                      transforms.RandomHorizontalFlip(p=0.5),
                                      transforms.ColorJitter(brightness=(0.5, 1),   #밝기
                                                             contrast = (0.5, 2),   #대비
                                                             saturation = (0.5, 2), #채도
                                                             hue = (-0.01, 0.1)),   #색조
                                      transforms.RandomResizedCrop((224, 224), scale=(0.1, 1), ratio=(0.5, 2)),
                                      transforms.ToTensor(),
                                      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

test_transform = transforms.Compose([transforms.Resize((224, 224)),
                                     transforms.ToTensor(),
                                     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_dataset = datasets.ImageFolder('/Users/mangodingo/Desktop/VS/Study/data/dataset_sample/train/', transform=train_transform)
test_dataset = datasets.ImageFolder('/Users/mangodingo/Desktop/VS/Study/data/dataset_sample/val/', transform=test_transform)

batch_size = 32

train_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle= True)
test_loader = DataLoader(train_dataset, batch_size= batch_size, shuffle= False)

In [3]:
model = models.resnet50(pretrained = True)
num_classes = 24
model.fc = nn.Linear(model.fc.in_features, num_classes)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
best_accuracy = 0.0  # 가장 높은 정확도 저장을 위한 변수 초기화

num_epochs = 2
total_accuracy = 0.0

# 평가 결과 저장을 위한 리스트
accuracy_list = []

# 모델 훈련
for epoch in range(num_epochs):
    model.train() # model.train()은 파이토치(PyTorch)에서 모델의 학습 모드를 설정하는 메소드
    correct = 0
    total = 0

    running_loss = 0.0 # loss 값
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs) # Resnet50 모델에 input
        _, predicted = torch.max(outputs.data, 1) # 확률이 제일 높은 output 선택

        loss = criterion(outputs, labels)
        loss.backward() # optimizer에게 loss function를 효율적으로 최소화 할 수 있게 파라미터 수정

        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {epoch_loss:.4f}")

    accuracy = 100.0 * correct / total

    total_accuracy += accuracy

    print(f"Train Accuracy: {accuracy:.2f}%")

    # 모델 평가
    model.eval() # model.eval()은 파이토치(PyTorch)에서 모델의 평가 모드를 설정하는 메소드
    correct = 0
    total = 0
    total_accuracy = 0.0

    with torch.no_grad(): # gradient를 트래킹하지 않음
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)

            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100.0 * correct / total

    total_accuracy += accuracy

    print(f"Test Accuracy: {accuracy:.2f}%")

    # 가장 높은 정확도를 가진 모델의 가중치 저장
    if accuracy > best_accuracy:
        best_accuracy = accuracy
        best_acc_weights = model.state_dict()
    accuracy_list.append(accuracy)

# 가장 높은 정확도와 f1 score를 가진 모델의 가중치 저장
torch.save(best_acc_weights, "/Users/mangodingo/Desktop/VS/Study/best_model_acc.pth")

print(f"Highest Test Accuracy: {best_accuracy:.2f}%")

Epoch [1/2], Loss: 1.9779
Train Accuracy: 34.22%
Test Accuracy: 31.22%
Epoch [2/2], Loss: 1.6080
Train Accuracy: 44.67%
Test Accuracy: 31.22%
Highest Test Accuracy: 31.22%
